# Vision-based Navigation - Day 2 - Practical Work 1
## Two-view relative navigation for vertical landing

In this exercise, we consider a vehicle attempting to land.

It has a vertical, downwards motion and a camera is pointed vertically, thus directly towards the targeted landing site.

<center><img src="img/scene.svg" style="width: 600px;"/></center>

This vehicle, which could be a planetary lander, a helicopter or a drone is using feature tracking aka optical flow in order to remain on its descent trajectory. We propose to verify that we are able to estimate its relative motion.

### Hypotheses
* An image processing algorithm from TESUR provides us with a list of tracked features at each camera sample time
* The cameras are already calibrated: we have access to the normalised coordinates of the features


### Case A: Standalone relative navigation

Eight features are successfully tracked and provided to the navigation algorithms.

We are given the normalised coordinates of these features in two successive views which create an unknown baseline $d\neq 0$. Their $(u,v,1)$ coordinates are the following, in rows of $(u,v)$:

$$\scriptsize\hat{X}_1 = \left[\begin{matrix}
0.033947250438723896 & 0.14096739588965254 \\
-0.05347924602006227 & 0.04427851552196613 \\
-0.09383302129855033 & -0.16176122076620428 \\
-0.08345493409585876 & 0.15021888137251296 \\
-0.18824546195400768 & -0.00978646132483708 \\
-0.149057329829307 & -0.0730898875997048 \\
0.13763868713597027 & -0.16528160338087666 \\
0.12370153058659338 & 0.06846735878978359
 \end{matrix}\right] \qquad \hat{X}_2 = \left[\begin{matrix}
0.07346125871049501 & 0.30505098956060833 \\
-0.11579494807054277 & 0.0958732365744898 \\
-0.202952166349311 & -0.3498745935224136 \\
-0.18054031393960415 & 0.32497256509121736 \\
-0.4071494279581669 & -0.02116678983271107 \\
-0.322482381846822 & -0.15812842661988288 \\
0.2975800223256499 & -0.35734504773001935 \\
0.267697819361328 & 0.14816763025114252
 \end{matrix}\right]$$
 
  #### **a)** Use the eight-point method to estimate the essential matrix $\tilde{E}$ that describes the two-view geometry. Use the SVD of $\tilde{E}$ to form $E$, the essential matrix that respects $E=U diag(1,1,0) V^T$ with $det(U V^T)=1$.
  
  _Note: An alternative would be using Nister's five point algorithm to estimate $E$, but that requires more lines of code than acceptable for this short exercise. If you're fast, you can check that your conclusions also hold with this alternative, for example by using ```cv::findEssentialMat()```._
  
  _Tip: Knowing $\hat{x}_{1}^T E \hat{x}_2 = 0$ for all $x$, we can estimate $E$ by solving the linear equation $De=0$ where:_
  
  $$ D = \left[\begin{matrix}
x_{1,11}x_{2,11} & x_{1,11}x_{2,12} & x_{1,11} & x_{1,12}x_{2,11} & x_{1,12}x_{2,12} & x_{1,12} & x_{2,11} & x_{2,12} & 1  \\
x_{1,21}x_{2,21} & x_{1,21}x_{2,22} & x_{1,21} & x_{1,22}x_{2,21} & x_{2,22}x_{2,22} & x_{1,22} & x_{2,21} & x_{2,22} & 1  \\
& \qquad\qquad\vdots \\
x_{1,81}x_{2,81} & x_{1,81}x_{2,82} & x_{1,81} & x_{1,82}x_{2,81} & x_{2,82}x_{2,82} & x_{1,82} & x_{2,81} & x_{2,82} & 1
 \end{matrix}\right]$$
 
 _and_
 
 $$ \tilde{e} = \left[\begin{matrix} \tilde{e}_{11} & \tilde{e}_{21} & \tilde{e}_{31} & \dots & \tilde{e}_{33} \end{matrix}\right]^T$$
 
 _is the essential vector (column representation of $\tilde{E}$) which can be found as the right null-space of $D$_

In [1]:
import numpy as np
# A matrix can be formed as follows
A = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
# and its SVD A=USV can be computed
U, S, V = np.linalg.svd(A)

# Our input matrices are copied here
x1 = np.array([[0.033947250438723896 , 0.14096739588965254],
              [-0.05347924602006227 , 0.04427851552196613],
              [-0.09383302129855033 , -0.16176122076620428],
              [-0.08345493409585876 , 0.15021888137251296],
              [-0.18824546195400768 , -0.00978646132483708],
              [-0.149057329829307 , -0.0730898875997048],
              [0.13763868713597027 , -0.16528160338087666],
              [0.12370153058659338 , 0.06846735878978359]])

x2 = np.array([[0.07346125871049501 , 0.30505098956060833],
              [-0.11579494807054277 , 0.0958732365744898],
              [-0.202952166349311 , -0.3498745935224136],
              [-0.18054031393960415 , 0.32497256509121736],
              [-0.4071494279581669 , -0.02116678983271107],
              [-0.322482381846822 , -0.15812842661988288],
              [0.2975800223256499 , -0.35734504773001935],
              [0.267697819361328 , 0.14816763025114252]])

  #### **b)** Use this essential matrix to compute the four $(R,t)$ motion solutions that respect the epipolar constraints. Find the correct $(R,t)$ solution by enforcing the chirality constraint. Is this result expected?
   _Tip: To enforce the positive depth aka chirality contraint, you need to derive how to triangulate points, i.e. find their depths in view 1 and 2 given known $(R,t)$ and $(x_{i,1},x_{i,2})$. Use the following notations:_
   * _$\rho_{i,1}$ (resp. $\rho_{i,2}$): the depth of point $i$ in frame 1 (resp. 2)_
   * _$\vec{l}_{i,1}^1$ (resp. $\vec{l}_{i,2}^1$): the line of sight that corresponds to the normalised coordinate $\vec{x}_{i,1}^1$ (resp. $\vec{x}_{i,2}^1$), given **in frame 1 attitude**_
   * _$\vec{t_{12}}$: the translation vector from frame 1 to frame 2_
   
_Indeed, since we know $R = R_{12}^{1}$, we can compute $\hat{X}^{1}_{2}$ which contains the coordinates of the points in frame $2$ with compensated rotation, i.e. as if the camera maintained constant attitude during the motion._

_Here's a sketch to get you started:_

<center><img src="img/epip.svg" alt="Drawing" style="width: 600px;"/></center>

We find that the unique solution which respects chirality is
  $$R = \left[\begin{matrix}
1 & 0 & 0 \\
0 & 1 & 0 \\
0 & 0 & 1
 \end{matrix}\right], \qquad t = \left[\begin{matrix}
0 & 0 & 1
 \end{matrix}\right]$$
minus numerical errors.

That sounds about right for a vehicle in strict vertical descent: the Z camera axis is aligned with the direction of translation.

**We observe that we are going in the right direction, but remember: we do not know how far we have moved because we can never observe the scale with optical flow: $\mathbf{t}$ remains up to scale.**

  #### **c)** At a different time, we receive these new measurements:
  $$\scriptsize\hat{X}_{1'} = \left[\begin{matrix}
-0.16384408602803172 & -0.28057420740421607 \\
0.11321415386483905 & 0.18072073008239153 \\
0.12305886289656569 & 0.04008202962915759 \\
0.24400814528634676 & 0.04430119064275458 \\
-0.25244646731356896 & 0.1947846001277149 \\
0.10055667082404805 & 0.09071196179232183 \\
-0.08508641377422074 & 0.07524170474246622 \\
0.1033694448331128 & 0.2285378882364913 
 \end{matrix}\right] \qquad \hat{X}_{2'} = \left[\begin{matrix}
-0.29010998526619014 & -0.49679778592792123 \\
0.20046226449719654 & 0.31999255885578815 \\
0.2178937657578245 & 0.07097111227538903 \\
0.4320522098169678 & 0.07844175567280098 \\
-0.4469934966118416 & 0.34489470351382806 \\
0.17805033430496067 & 0.16061883304433255 \\
-0.15065797518116641 & 0.1332264739204888 \\
0.18303076323656856 & 0.40465985069312377
 \end{matrix}\right]$$
  #### Compute the new $E$ as well as the new $R$ and $t$ solutions. Is this result expected?

In [152]:
# Our new input matrices are copied here
x1 = np.array([[-0.16384408602803172 , -0.28057420740421607],
              [0.11321415386483905 , 0.18072073008239153],
              [0.12305886289656569 , 0.04008202962915759],
              [0.24400814528634676 , 0.04430119064275458],
              [-0.25244646731356896 , 0.1947846001277149],
              [0.10055667082404805 , 0.09071196179232183],
              [-0.08508641377422074 , 0.07524170474246622],
              [0.1033694448331128 , 0.2285378882364913]])

x2 = np.array([[-0.29010998526619014 , -0.49679778592792123],
              [0.20046226449719654 , 0.31999255885578815],
              [0.2178937657578245 , 0.07097111227538903],
              [0.4320522098169678 , 0.07844175567280098],
              [-0.4469934966118416 , 0.34489470351382806],
              [0.17805033430496067 , 0.16061883304433255],
              [-0.15065797518116641 , 0.1332264739204888],
              [0.18303076323656856 , 0.40465985069312377]])

# Notebooks are incremental: simply run this cell and your previous calculation cells above to compute the new results. No need to c/c :)

### Case B: Gyro-aided relative navigation

We now have access to gyro measurements from which we know the rotation increment between frames $1'$ and $2'$:

$$R_{1'\rightarrow 2'}^{1'} = \left[\begin{matrix}
  1 & 0 & 0 \\
  0 & 1 & 0 \\
  0 & 0 & 1
  \end{matrix}\right]$$
  
  #### **a)** Using basic projective geometry and the minimum number of point correspondences , find the expression of the translation direction $\vec{t_d}$ knowing $R$. How many points are needed?
_Tip: Keep the notations from Case A._

  #### **b)** Compute the translation direction using the minimum number of point correspondences from _Case A, **c)**_. What can you conclude with regard to the implementation of _Case A_?